In [26]:
import pyodbc
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

server = 'mia10.database.windows.net'
database = 'mia10_db'
username = 'user_reader'
password = '7R&o&o4#~756^z'
driver = '{ODBC Driver 17 for SQL Server}'

connection_string = f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}'

conn = pyodbc.connect(connection_string)

query = '''
SELECT m.slug_game, m.event_title, m.medal_type, c.country_name, e.game_year
FROM medals m
JOIN events e ON LOWER(REPLACE(m.slug_game, '-', ' ')) = LOWER(e.game_name)
JOIN country c ON m.athlete_country_id = c.country_id
WHERE e.game_season = 'Summer'
'''

df = pd.read_sql(query, conn, index_col=None)

conn.close()

df.head(10)


/tmp/ipykernel_299921/3551398233.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col=None)


,slug_game,event_title,medal_type,country_name,game_year
0,munich-1972,twoperson keelboat open Star mixed,SILVER,Sweden,1972
1,munich-1972,twoperson keelboat open Star mixed,SILVER,Sweden,1972
2,montreal-1976,tempest mixed,GOLD,Sweden,1976
3,montreal-1976,tempest mixed,GOLD,Sweden,1976
4,los-angeles-1932,twoperson keelboat open Star mixed,BRONZE,Sweden,1932
5,los-angeles-1984,K2 500m kayak double men,SILVER,Sweden,1984
6,los-angeles-1984,K2 500m kayak double men,SILVER,Sweden,1984
7,los-angeles-1984,K1 500m kayak single men,SILVER,Sweden,1984
8,los-angeles-1984,K2 500m kayak double women,GOLD,Sweden,1984
9,los-angeles-1984,K1 500m kayak single women,GOLD,Sweden,1984


In [27]:
countries_rio_2016 = df[df['slug_game'] == 'rio-2016']['country_name'].unique()
df_filtered = df[df['country_name'].isin(countries_rio_2016)]

# grouper par pays, édition et type de médaille
grouped = df_filtered.groupby(['country_name', 'slug_game', 'medal_type']).size().unstack(fill_value=0)

grouped.columns = [f'{col}_count' for col in grouped.columns]

grouped.reset_index(inplace=True)

grouped.head(1000)

,country_name,slug_game,BRONZE_count,GOLD_count,SILVER_count
0,Argentina,athens-2004,4,0,0
1,Argentina,atlanta-1996,0,0,1
2,Argentina,barcelona-1992,2,0,0
3,Argentina,beijing-2008,2,2,0
4,Argentina,berlin-1936,2,0,0
...,...,...,...,...,...
422,United States of America,rome-1960,2,0,0
423,United States of America,seoul-1988,4,9,6
424,United States of America,sydney-2000,6,9,6
425,United States of America,tokyo-1964,3,0,7


In [28]:
# Encodage des noms de pays
encoder = OneHotEncoder(sparse_output=False)
country_encoded = encoder.fit_transform(grouped[['country_name']])
country_encoded_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country_name']))

grouped = pd.concat([grouped, country_encoded_df], axis=1) # Ajouter les colonnes encodées au DataFrame

# caractéristiques supplémentaires
grouped['total_medals'] = grouped['BRONZE_count'] + grouped['GOLD_count'] + grouped['SILVER_count']
grouped['gold_silver_diff'] = grouped['GOLD_count'] - grouped['SILVER_count']
grouped['gold_bronze_diff'] = grouped['GOLD_count'] - grouped['BRONZE_count']

# préparation des données pour random forest
def create_sequences(data, time_steps=3):
    Xs, ys = [], []
    for i in range(len(data) - time_steps):
        Xs.append(data.iloc[i:(i + time_steps), 2:].values)
        ys.append(data.iloc[i + time_steps, [3, 4, 2]].values) # assuming [3, 4, 2] are the target columns
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)

# séquences
time_steps = 3
X_seq, y_seq = create_sequences(grouped, time_steps)

# Standardiser les données
scaler = StandardScaler()
X_seq_shape = X_seq.shape
X_seq = scaler.fit_transform(X_seq.reshape(-1, X_seq.shape[-1])).reshape(X_seq_shape)

X_seq = X_seq.reshape(X_seq.shape[0], -1) # Aplatir les données

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

def objective(trial):
    # hyperparamètres
    n_estimators = trial.suggest_int('n_estimators', 100, 300)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    max_depth = trial.suggest_int('max_depth', 10, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print(f'Best Hyperparameters: {study.best_params}')

best_model = RandomForestRegressor(
    **study.best_params,
    random_state=42
)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# évaluer
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

[I 2024-05-30 16:30:13,936] A new study created in memory with name: no-name-095fd031-f993-4ee5-9266-a7f0298722e6
[I 2024-05-30 16:30:14,009] Trial 0 finished with value: 5.706991283260668 and parameters: {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 5.706991283260668.
[I 2024-05-30 16:30:14,205] Trial 1 finished with value: 5.599016473121143 and parameters: {'n_estimators': 250, 'max_features': 'sqrt', 'max_depth': 26, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 1 with value: 5.599016473121143.
[I 2024-05-30 16:30:14,342] Trial 2 finished with value: 5.450252538279423 and parameters: {'n_estimators': 188, 'max_features': 'sqrt', 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 2 with value: 5.450252538279423.
[I 2024-05-30 16:30:14,453] Trial 3 finished with value: 5.869683155381935 and parameters: {'n_estimators': 144, 'max_features': 'log2', 'm

Best Hyperparameters: {'n_estimators': 235, 'max_features': 'log2', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1}
Mean Squared Error: 5.2861613829730345
R^2 Score: 0.2239105739250845


In [29]:
def predict_medals_next_edition(country, model, scaler, data, encoder, time_steps=3):
    # encodage du pays
    country_encoded = encoder.transform([[country]])
    country_encoded_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country_name']))
    
    # Sélectionner les données du pays
    country_data = data[data['country_name'] == country].sort_values(by='slug_game').iloc[-time_steps:, 2:]
    country_data = pd.concat([country_data.reset_index(drop=True), country_encoded_df], axis=1)
    
    if country_data.shape[0] < time_steps:
        raise ValueError(f"Not enough data for {country} to create a prediction with {time_steps} time steps.")
    
    # Standardiser les données
    expected_features = scaler.n_features_in_
    current_features = country_data.shape[1]
    
    if current_features < expected_features:
        # ajout des 0 si nécessaire
        missing_features = expected_features - current_features
        country_data = np.hstack([country_data.values, np.zeros((country_data.shape[0], missing_features))])
    elif current_features > expected_features:
        # sup des colonnes si nécessaire
        country_data = country_data.iloc[:, :expected_features]

    country_data = scaler.transform(country_data)
    
    # aplatir les données
    country_data = country_data.reshape(1, -1)
    
    predicted_medals = model.predict(country_data)
    return pd.DataFrame(predicted_medals, columns=['GOLD_count', 'SILVER_count', 'BRONZE_count'])

In [30]:
def predict_all_countries_next_edition(model, scaler, data, encoder, time_steps=3):
    unique_countries = data['country_name'].unique()
    predictions = []

    for country in unique_countries:
        try:
            predicted_medals = predict_medals_next_edition(country, model, scaler, data, encoder, time_steps)
            predicted_medals['country_name'] = country
            predictions.append(predicted_medals)
        except ValueError as e:
            print(f"Skipping country {country}: {e}")

    predictions_df = pd.concat(predictions).reset_index(drop=True)
    return predictions_df

predictions_df = predict_all_countries_next_edition(best_model, scaler, grouped, encoder)

total_predicted_medals = predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum().sum()

# calculer le facteur de normalisation pour ajuster les médailles à la somme cible (987)
medal_target = 987
normalization_factor = medal_target / total_predicted_medals

# appliquer aux prédictions
predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']] *= normalization_factor

# arrondi entier le plus proche
predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']] = predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']].round().astype(int)

predictions_df['total_medals'] = predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum(axis=1)

# sort par nombre total de médailles si égalité prendre or
predictions_df = predictions_df.sort_values(by=['total_medals', 'GOLD_count'], ascending=False)

predictions_df.head(50)

/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but Sta

Skipping country Lithuania: Not enough data for Lithuania to create a prediction with 3 time steps.


/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/mory/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but Sta

,GOLD_count,SILVER_count,BRONZE_count,country_name,total_medals
22,45,14,14,Norway,73
23,22,17,17,People's Republic of China,56
35,22,18,15,United States of America,55
10,15,17,16,Germany,48
25,11,18,15,ROC,44
0,11,9,15,Argentina,35
27,10,13,11,Romania,34
11,7,14,13,Great Britain,34
31,7,15,6,South Africa,28
28,8,8,10,Russian Federation,26
